In [4]:
import sys, os
from Deconvolution.BLADE import Framework
import numpy as np
from numpy import transpose as t
import itertools
import pickle
from scipy.optimize import nnls
from sklearn.svm import SVR
from sklearn.svm import NuSVR
from sklearn.metrics import mean_squared_error as mse
import pandas as pd

# modules for visualization
import qgrid
from matplotlib import pyplot as plt
import seaborn as sns

### Run BLADE with TCGA bulk and Puram scRNA-seq reference


#### Application of deconvolution methods

From here, we will apply the following three methods for further performance comparison:
1. BLADE (estimation of cellular fraction + group-mode/high-resolution-mode purification)
2. NNLS (estimation of fraction)
3. SVR followed by NNLS (estimation of fraction + group-mode purification) - similar to CIBERSORTx


##### 1. Application of BLADE

These are the key parameters used in BLADE (note that there is default settings of these parameters, if not specified):
- Hyperparameters (`hyperpars`): `Alpha`, `Alpha0`, `Kappa0` and `SigmaY`, each of which can be defined as a list of options. BLADE takes an empirical Bayes approach to find the optimal parameter set given the all possible combinations. 
- `Nrep`: Number of repeat for evaluating each parameter configuration.
- `Nrepfinal`: Number of repeated optimizations for the final parameter set.
- `Njob`: Number of parallel jobs.

In [75]:
hyperpars = {
    'Alpha': [1, 10],
    'Alpha0': [0.1, 0.5, 1, 5, 10],
    'Kappa0': [1, 0.5, 0.1],
    'SY': np.sqrt([0.1, 0.5, 1, 1.5, 2])
}

Nrep=3
Nrepfinal=10
Njob=20

In [76]:
# df_Puram_std = pd.read_csv("/home/cke/Puram/HNSCC2PuramGSE103322_HNSCC_exp_std.tsv",sep='\t',index_col=0)
# df_Puram_mean = pd.read_csv("/home/cke/Puram/HNSCC2PuramGSE103322_HNSCC_exp_mean.tsv",sep='\t',index_col=0)

# merged all tumor cell types
df_Puram_std = pd.read_csv("/home/cke/Puram/HNSCC2PuramGSE103322_HNSCC_exp_std_simple.tsv",sep='\t',index_col=0)
df_Puram_mean = pd.read_csv("/home/cke/Puram/HNSCC2PuramGSE103322_HNSCC_exp_mean_simple.tsv",sep='\t',index_col=0)

In [77]:
df_Puram_std

,B cell,Dendritic,Endothelial,Fibroblast,Macrophage,Mast,NA,T cell,myocyte,tumor
ENSG00000121410,1.083922,0.818420,0.399890,0.801904,0.564758,0.838012,0.614700,0.668682,0.000000,0.544478
ENSG00000268895,0.511951,0.016481,0.386574,0.758462,0.462413,1.065588,0.401408,0.406487,0.000000,0.499750
ENSG00000148584,0.006775,0.018113,0.035513,0.017553,0.003137,0.030807,0.024420,0.022276,0.000000,0.012275
ENSG00000175899,0.108961,0.651788,3.116251,3.586992,2.773042,0.804513,2.095236,0.318067,2.588845,0.313355
ENSG00000245105,0.089944,0.028935,0.747009,0.484618,0.415440,0.724833,0.226807,0.407835,0.000000,0.324657
...,...,...,...,...,...,...,...,...,...,...
ENSG00000203995,0.362794,0.374291,0.219057,0.196687,0.118434,0.137730,0.457341,0.240729,0.266944,0.254686
ENSG00000162378,0.457661,0.168491,0.897044,0.904963,0.659679,0.528316,0.632295,0.442593,0.147025,0.536564
ENSG00000159840,0.923420,2.148970,2.143931,2.233748,2.064606,1.246275,1.610354,1.907724,1.370751,1.663473
ENSG00000074755,0.630889,0.826259,0.918515,0.843143,0.451562,0.614904,0.610981,0.911177,0.707439,0.588526


In [78]:
df_Puram_mean

,B cell,Dendritic,Endothelial,Fibroblast,Macrophage,Mast,NA,T cell,myocyte,tumor
ENSG00000121410,0.531012,0.306200,0.077734,0.256194,0.219202,0.305957,0.151037,0.124142,0.000000,0.197286
ENSG00000268895,0.091276,0.002308,0.054535,0.153220,0.078708,0.238673,0.057073,0.036260,0.000000,0.112074
ENSG00000148584,0.000998,0.005308,0.004459,0.002982,0.000482,0.007019,0.007832,0.007393,0.000000,0.002747
ENSG00000175899,0.021167,0.137365,6.034123,5.435018,2.885008,0.259597,0.708786,0.079807,3.576743,0.031501
ENSG00000245105,0.007657,0.004052,0.171338,0.077473,0.057223,0.105998,0.021405,0.030704,0.000000,0.038320
...,...,...,...,...,...,...,...,...,...,...
ENSG00000203995,0.206693,0.230500,0.215373,0.172395,0.091906,0.180291,0.333693,0.335935,0.180180,0.134803
ENSG00000162378,0.223046,0.182567,0.487771,0.448538,0.316599,0.268201,0.374778,0.341616,0.115098,0.285960
ENSG00000159840,0.215561,1.949729,1.787872,1.973410,2.810183,0.403501,0.868257,0.814331,0.739055,1.611830
ENSG00000074755,0.165944,0.254342,0.334769,0.280260,0.149263,0.143537,0.181321,0.233642,0.357566,0.250754


In [15]:
df_TCGA = pd.read_csv("/home/cke/TCGA-HNSC.htseq_counts_exp2.tsv",sep='\t',index_col=0)

In [79]:
df_TCGA

,TCGA-BB-4224-01A,TCGA-H7-7774-01A,TCGA-CV-6943-01A,TCGA-CN-5374-01A,TCGA-CQ-6227-01A,TCGA-CV-6959-01A,TCGA-F7-A61V-01A,TCGA-CV-7413-01A,TCGA-CV-7247-01A,TCGA-CR-5249-01A,...,TCGA-CV-6960-11A,TCGA-CV-A464-01A,TCGA-C9-A47Z-01A,TCGA-CN-6010-01A,TCGA-WA-A7GZ-11A,TCGA-CV-7235-01A,TCGA-CX-7086-01A,TCGA-CV-6935-11A,TCGA-P3-A6SW-01A,TCGA-HD-A6HZ-01A
Ensembl_ID,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,2237.0,2740.0,2686.0,2086.0,10167.0,1154.0,1978.0,1930.0,1066.0,2538.0,...,3340.0,929.0,1707.0,2218.0,2537.0,492.0,2741.0,8492.0,770.0,923.0
ENSG00000000005,2.0,0.0,0.0,1.0,9.0,6.0,1.0,1.0,1.0,0.0,...,0.0,2.0,0.0,0.0,41.0,0.0,0.0,3.0,1.0,0.0
ENSG00000000419,1606.0,1691.0,1649.0,2333.0,3021.0,2766.0,1762.0,1668.0,1760.0,1268.0,...,1388.0,2332.0,1926.0,1574.0,1171.0,976.0,1952.0,1578.0,1569.0,1183.0
ENSG00000000457,1063.0,803.0,917.0,1288.0,537.0,527.0,482.0,671.0,600.0,1066.0,...,608.0,472.0,410.0,852.0,557.0,376.0,1003.0,818.0,620.0,607.0
ENSG00000000460,1208.0,317.0,402.0,1105.0,459.0,747.0,331.0,476.0,874.0,950.0,...,206.0,386.0,210.0,729.0,127.0,363.0,1103.0,281.0,591.0,344.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSGR0000275287,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSGR0000276543,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSGR0000277120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [83]:
merge_genes_mean = pd.merge(df_Puram_mean,df_TCGA,left_index=True,right_index=True,how='inner')
merge_genes_std = pd.merge(df_Puram_std,df_TCGA,left_index=True,right_index=True,how='inner')
#21706 genes in common
# df_TCGA_shared = merge_genes_mean.iloc[:,24:]
# df_shared_mean = merge_genes_mean.iloc[:,:24]
# df_shared_std = merge_genes_std.iloc[:,:24]

#simple tumor cell type setup
df_TCGA_shared = merge_genes_mean.iloc[:,10:]
df_shared_mean = merge_genes_mean.iloc[:,:10]
df_shared_std = merge_genes_std.iloc[:,:10]

In [84]:
df_TCGA_shared

,TCGA-BB-4224-01A,TCGA-H7-7774-01A,TCGA-CV-6943-01A,TCGA-CN-5374-01A,TCGA-CQ-6227-01A,TCGA-CV-6959-01A,TCGA-F7-A61V-01A,TCGA-CV-7413-01A,TCGA-CV-7247-01A,TCGA-CR-5249-01A,...,TCGA-CV-6960-11A,TCGA-CV-A464-01A,TCGA-C9-A47Z-01A,TCGA-CN-6010-01A,TCGA-WA-A7GZ-11A,TCGA-CV-7235-01A,TCGA-CX-7086-01A,TCGA-CV-6935-11A,TCGA-P3-A6SW-01A,TCGA-HD-A6HZ-01A
ENSG00000121410,30.0,0.0,33.0,20.0,17.0,9.0,2.0,38.0,3.0,6.0,...,1.0,9.0,5.0,9.0,6.0,29.0,6.0,1.0,17.0,10.0
ENSG00000268895,78.0,17.0,174.0,124.0,271.0,73.0,30.0,249.0,24.0,67.0,...,11.0,126.0,75.0,49.0,26.0,106.0,61.0,14.0,92.0,33.0
ENSG00000148584,4.0,0.0,1.0,4.0,3.0,2.0,1.0,3.0,0.0,0.0,...,1.0,2.0,0.0,1.0,4.0,0.0,1.0,0.0,1.0,15.0
ENSG00000175899,2959.0,3928.0,28417.0,6318.0,66382.0,29210.0,4738.0,8560.0,22096.0,12114.0,...,5405.0,17577.0,2659.0,14591.0,12013.0,18501.0,6783.0,9184.0,4457.0,11552.0
ENSG00000245105,25.0,8.0,33.0,22.0,45.0,16.0,5.0,26.0,22.0,27.0,...,11.0,39.0,10.0,35.0,54.0,28.0,16.0,18.0,15.0,46.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000203995,379.0,69.0,6.0,404.0,95.0,5.0,46.0,5.0,168.0,386.0,...,27.0,19.0,10.0,16.0,108.0,382.0,545.0,61.0,378.0,51.0
ENSG00000162378,1855.0,2113.0,1915.0,1248.0,2658.0,2176.0,991.0,1421.0,1738.0,2397.0,...,1654.0,1491.0,1262.0,2294.0,4495.0,1839.0,2212.0,2085.0,1011.0,1961.0
ENSG00000159840,2775.0,12997.0,21546.0,5731.0,21868.0,25870.0,5741.0,9615.0,11078.0,14149.0,...,12796.0,14212.0,7686.0,11467.0,2721.0,20198.0,14575.0,10871.0,5005.0,5230.0
ENSG00000074755,2025.0,4457.0,5918.0,2193.0,4056.0,5792.0,1852.0,3684.0,3912.0,4919.0,...,3852.0,2396.0,2851.0,4904.0,3292.0,3774.0,3315.0,5258.0,1394.0,3245.0


In [85]:
df_shared_mean

,B cell,Dendritic,Endothelial,Fibroblast,Macrophage,Mast,NA,T cell,myocyte,tumor
ENSG00000121410,0.531012,0.306200,0.077734,0.256194,0.219202,0.305957,0.151037,0.124142,0.000000,0.197286
ENSG00000268895,0.091276,0.002308,0.054535,0.153220,0.078708,0.238673,0.057073,0.036260,0.000000,0.112074
ENSG00000148584,0.000998,0.005308,0.004459,0.002982,0.000482,0.007019,0.007832,0.007393,0.000000,0.002747
ENSG00000175899,0.021167,0.137365,6.034123,5.435018,2.885008,0.259597,0.708786,0.079807,3.576743,0.031501
ENSG00000245105,0.007657,0.004052,0.171338,0.077473,0.057223,0.105998,0.021405,0.030704,0.000000,0.038320
...,...,...,...,...,...,...,...,...,...,...
ENSG00000203995,0.206693,0.230500,0.215373,0.172395,0.091906,0.180291,0.333693,0.335935,0.180180,0.134803
ENSG00000162378,0.223046,0.182567,0.487771,0.448538,0.316599,0.268201,0.374778,0.341616,0.115098,0.285960
ENSG00000159840,0.215561,1.949729,1.787872,1.973410,2.810183,0.403501,0.868257,0.814331,0.739055,1.611830
ENSG00000074755,0.165944,0.254342,0.334769,0.280260,0.149263,0.143537,0.181321,0.233642,0.357566,0.250754


In [86]:
df_shared_std

,B cell,Dendritic,Endothelial,Fibroblast,Macrophage,Mast,NA,T cell,myocyte,tumor
ENSG00000121410,1.083922,0.818420,0.399890,0.801904,0.564758,0.838012,0.614700,0.668682,0.000000,0.544478
ENSG00000268895,0.511951,0.016481,0.386574,0.758462,0.462413,1.065588,0.401408,0.406487,0.000000,0.499750
ENSG00000148584,0.006775,0.018113,0.035513,0.017553,0.003137,0.030807,0.024420,0.022276,0.000000,0.012275
ENSG00000175899,0.108961,0.651788,3.116251,3.586992,2.773042,0.804513,2.095236,0.318067,2.588845,0.313355
ENSG00000245105,0.089944,0.028935,0.747009,0.484618,0.415440,0.724833,0.226807,0.407835,0.000000,0.324657
...,...,...,...,...,...,...,...,...,...,...
ENSG00000203995,0.362794,0.374291,0.219057,0.196687,0.118434,0.137730,0.457341,0.240729,0.266944,0.254686
ENSG00000162378,0.457661,0.168491,0.897044,0.904963,0.659679,0.528316,0.632295,0.442593,0.147025,0.536564
ENSG00000159840,0.923420,2.148970,2.143931,2.233748,2.064606,1.246275,1.610354,1.907724,1.370751,1.663473
ENSG00000074755,0.630889,0.826259,0.918515,0.843143,0.451562,0.614904,0.610981,0.911177,0.707439,0.588526


Given the configuration above, BLADE is applied to each of the simulation dataset created previously.  

BLADE produce several outcomes:
- `final_obj`: final BLADE object with optimized variational parameters
- `best_obj`: BLADE object trained with the best parameter set found by the Empirical Bayes framework. Empirical Bayes framework is applied after selecting a subset of samples (5 samples; indicated by `Ind_sample` below), and thus the outcome contains only 5 samples. If `Nsample` <= 5, `final_obj` is identical to `best_obj`.
- `best_set`: Best parameter set defined by Empirical Bayes framework.
- `outs`: Outcome of BLADE for every possible combination of hyperparameters, used in the Empirical Bayes framework. 


- There are nan in mean and std matrix! NAs are filled with 0?

full tumor type setup:
- ngenes = 21706 common genes
- ncells = 24, including all 16 tumor types
- nsample = 546

simple tumor type setup:
- ngenes = 21706 common genes
- ncells = 10, all tumor types are merged
- nsample = 546

In [87]:
np.shape(np.log(df_shared_mean).to_numpy())

/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


(21706, 10)

In [88]:
np.log(df_shared_mean)

,B cell,Dendritic,Endothelial,Fibroblast,Macrophage,Mast,NA,T cell,myocyte,tumor
ENSG00000121410,-0.632970,-1.183518,-2.554465,-1.361819,-1.517760,-1.184311,-1.890229,-2.086331,-inf,-1.623102
ENSG00000268895,-2.393862,-6.071442,-2.908920,-1.875879,-2.542014,-1.432662,-2.863423,-3.317046,-inf,-2.188598
ENSG00000148584,-6.910022,-5.238551,-5.412856,-5.815101,-7.636877,-4.959173,-4.849494,-4.907282,-inf,-5.897214
ENSG00000175899,-3.855300,-1.985116,1.797431,1.692863,1.059528,-1.348625,-0.344202,-2.528144,1.274453,-3.457748
ENSG00000245105,-4.872197,-5.508603,-1.764118,-2.557825,-2.860798,-2.244336,-3.844143,-3.483347,-inf,-3.261771
...,...,...,...,...,...,...,...,...,...,...
ENSG00000203995,-1.576518,-1.467503,-1.535385,-1.757965,-2.386993,-1.713183,-1.097534,-1.090838,-1.713801,-2.003941
ENSG00000162378,-1.500378,-1.700638,-0.717909,-0.801762,-1.150120,-1.316019,-0.981422,-1.074069,-2.161969,-1.251902
ENSG00000159840,-1.534510,0.667690,0.581026,0.679763,1.033250,-0.907576,-0.141268,-0.205388,-0.302383,0.477370
ENSG00000074755,-1.796107,-1.369075,-1.094315,-1.272039,-1.902048,-1.941160,-1.707484,-1.453964,-1.028435,-1.383283


In [89]:
np.shape(df_shared_std.sample(n=1000).to_numpy())

(1000, 10)

In [58]:
df_shared_std.sample(n=1000)

,B cell,Dendritic,Endothelial,Fibroblast,Macrophage,Mast,T cell,myocyte,tumor 0,tumor 10,...,tumor 18,tumor 20,tumor 22,tumor 24,tumor 25,tumor 26,tumor 28,tumor 5,tumor 6,tumor 7
ENSG00000151962,0.000000,0.000000,0.016981,0.000000,0.000000,0.000000,0.083054,0.000000,0.000000,NaN,...,0.257166,0.000000,0.000000,0.000000,0.005815,0.064534,0.000000,0.000000,0.000000,0.000000
ENSG00000122490,0.486927,0.000000,0.927466,0.723861,0.987848,0.771898,0.618093,0.798420,0.868004,NaN,...,0.740991,0.600934,0.388827,0.000000,0.797882,0.525653,0.712201,0.761077,0.733606,0.000000
ENSG00000185085,0.601005,0.018887,0.917203,0.788548,0.678804,0.762877,0.584534,1.485492,1.068386,NaN,...,0.472561,0.991797,1.029212,0.504227,0.764847,1.178105,1.167443,1.019610,0.816560,0.000000
ENSG00000204070,1.677479,0.825748,1.193927,1.491258,1.464501,0.943033,1.372801,0.834716,1.170429,NaN,...,1.340436,1.102000,1.415156,0.551339,1.515581,1.572199,1.201452,1.197060,0.963107,0.000000
ENSG00000101457,1.224483,2.391255,1.471225,1.056877,1.439723,1.412687,0.895587,1.561137,1.654592,NaN,...,1.847418,1.626734,1.560493,1.323371,1.594807,1.592971,1.240978,1.612957,1.505847,2.285578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000165525,1.641139,2.132781,1.956707,1.771684,1.670853,1.864712,2.006128,1.703416,1.885736,NaN,...,1.504263,1.607155,1.510850,1.325304,1.385887,1.419300,1.055591,1.220961,1.358234,1.810530
ENSG00000167619,0.000000,0.000000,0.159955,0.049397,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,...,0.000000,0.007495,0.000000,0.000000,0.000000,0.013821,0.000000,0.000000,0.151951,0.000000
ENSG00000196872,0.000000,0.000000,0.659972,0.126939,0.000000,0.000000,0.034568,0.000000,0.000000,NaN,...,0.326633,0.029499,0.185246,0.666547,0.082394,0.280946,0.071069,0.364336,0.044418,0.000000
ENSG00000138738,0.298595,0.000000,0.784907,0.702185,0.057821,0.455345,0.271553,0.540022,0.713946,NaN,...,0.360076,0.494711,0.676880,0.045875,0.567114,0.493294,0.521035,0.421779,0.438032,0.000000


In [49]:
np.shape(df_TCGA_shared.to_numpy())

/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


(21706, 546)

In [90]:
Y = df_TCGA_shared.to_numpy()
mean = np.log(df_shared_mean).to_numpy() #highly suspect if log transform should be done here
sd = df_shared_std.to_numpy()

outfile = './BLADE/data/PuramTCGA.pickle'

final_obj, best_obj, best_set, outs = Framework(
    mean, sd, Y,
    Alphas=hyperpars['Alpha'], Alpha0s=hyperpars['Alpha0'], 
    Kappa0s=hyperpars['Kappa0'], SYs=hyperpars['SY'],
    Nrep=Nrep, Njob=Njob, Nrepfinal=Nrepfinal)

pickle.dump(
    {
        'final_obj': final_obj,
        'best_obj': best_obj,
        'best_set': best_set,
        'outs' : outs
    }, open(outfile, 'wb')
    )

/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


all of 21706 genes are used for optimization.
All samples are used during the optimization.
Initialization with Support vector regression


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:  2.3min
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:  2.4min
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:  4.6min
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:  4.7min
[Parallel(n_jobs=20)]: Done  45 tasks      | elapsed:  6.9min
[Parallel(n_jobs=20)]: Done  58 tasks      | elapsed:  7.0min
[Parallel(n_jobs=20)]: Done  73 tasks      | elapsed:  9.3min
[Parallel(n_jobs=20)]: Done  88 tasks      | elapsed: 11.5min
[Parallel(n_jobs=20)]: Done 105 tasks      | elapsed: 13.8min
[Parallel(n_jobs=20)]: Done 122 tasks      | elapsed: 16.0min
[Parallel(n_jobs=20)]: Done 141 tasks      | elapsed: 18.1min
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed: 18.6min
[Parallel(n_jobs=20)]: Done 181 tasks      | elapsed: 22.7min
[Parallel(n_jobs=20)]: Done 202 tasks      | elapsed: 25.0min
[Parallel(n_jobs=20)]: Done 225 tasks      | elapsed: 2

KeyboardInterrupt: 

### test run with a small subset of scrna reference, take randomly 1000 genes from both ref and bulk counts
> reduced hyperparam combinations

In [74]:
hyperpars_test = {
    'Alpha': [1, 10],
    'Alpha0': [0.1, 1, 10],
    'Kappa0': [1, 0.5, 0.1],
    'SY': np.sqrt([0.1, 1, 2])
}

df_mean_sample = df_shared_mean.sample(n=1000)
df_std_sample = df_shared_std[df_shared_std.index.isin(df_mean_sample.index)]
Y_sample = df_TCGA_shared[df_TCGA_shared.index.isin(df_mean_sample.index)]

Y = Y_sample.to_numpy()
mean = np.log(df_mean_sample).to_numpy() #highly suspect if log transform should be done here
sd = df_std_sample.to_numpSCy()

outfile = './BLADE/data/PuramTCGA_BLADE_SampleR1000.pickle'

final_obj, best_obj, best_set, outs = Framework(
    mean, sd, Y,
    Alphas=hyperpars_test['Alpha'], Alpha0s=hyperpars_test['Alpha0'], 
    Kappa0s=hyperpars_test['Kappa0'], SYs=hyperpars_test['SY'],
    Nrep=Nrep, Njob=10, Nrepfinal=Nrepfinal)

pickle.dump(
    {
        'final_obj': final_obj,
        'best_obj': best_obj,
        'best_set': best_set,
        'outs' : outs
    }, open(outfile, 'wb')
    )


all of 1000 genes are used for optimization.
All samples are used during the optimization.
Initialization with Support vector regression


/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   5 tasks      | elapsed:    6.7s
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:    7.0s
[Parallel(n_jobs=10)]: Done  21 tasks      | elapsed:    7.2s
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    7.4s
[Parallel(n_jobs=10)]: Done  41 tasks      | elapsed:    7.7s
[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed:    8.0s
[Parallel(n_jobs=10)]: Done  65 tasks      | elapsed:    8.3s
[Parallel(n_jobs=10)]: Done  78 tasks      | elapsed:    8.5s
[Parallel(n_jobs=10)]: Done  93 tasks      | elapsed:    8.9s
[Parallel(n_jobs=10)]: Done 108 tasks      | elapsed:    9.2s
[Parallel(n_jobs=10)]: Done 125 tasks      | elapsed:    9.7s
[Parallel(n_jobs=10)]: Done 142 tasks      | elapsed: 

No feature filtering is done (fsel = 0)


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:809: RuntimeWarning: divide by zero encountered in log
  return -self.Nsample*np.sum(np.log(Omega))
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:809: RuntimeWarning: divide by zero encountered in log
  return -self.Nsample*np.sum(np.log(Omega))
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:809: RuntimeWarning: divide by zero encountered in log
  return -self.Nsample*np.sum(np.log(Omega))
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:809: RuntimeWarning: divide by zero encountered in log
  return -self.Nsample*np.sum(np.log(Omega))
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:809: RuntimeWarning: divide by zero encountered in log
  return -self.Nsample*np.sum(np.log(Omega))
/ho

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGSEGV(-11)}